<a href="https://colab.research.google.com/github/moustafa-7/League-of-legends_Bigdata/blob/main/Bigdata_miniprojects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [194]:
import findspark
findspark.init()
import pyspark
import random

In [195]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LOL").getOrCreate()
sc = spark.sparkContext
d = spark.read.json("drive/MyDrive/small_matches.json", multiLine=True)

In [196]:
no_matches = d.count()

## 1. Champions win - pick - ban rates

In [264]:
def champions(row):
  c = []
  bans = []
  for p in row["participants"]:
    if(p["teamId"] == "Blue"): team_number = 0
    else: team_number = 1
    c.append((p["championId"], row["teams"][team_number]["win"]))
  
  for t in row["teams"]:
    for b in t["bans"]:
      if(b["championId"] != None):
        c.append((b["championId"],"ban"))
      # else:
      #   c.append((b["championid"],"ban"))

  return c

In [265]:
champions = d.rdd.flatMap(champions)

In [266]:
import time
def lookup(champions):
  for c in champions.collect():
    print(c)
    time.sleep(0.1)

In [267]:
by_champion = champions.groupByKey().mapValues(list)

In [268]:
# for c, val in by_champion.collect():
#     print(c,val)
#     time.sleep(0.1)

In [276]:
def compute_champion_rates(val):
    ban_count = val.count("ban")
    win_count = val.count("Win")
    lose_count = val.count("Fail")
    total_played_games = win_count+lose_count
    if(total_played_games):
      return [ban_count/no_matches, win_count/total_played_games, lose_count/total_played_games]
    else:
      return [ban_count/no_matches, "never_played", "never_played"]

In [277]:
win_pick_ban_rates = by_champion.map(lambda x: (x[0], compute_champion_rates(x[1])))

In [ ]:
for c in win_pick_ban_rates.collect():
    print(c)
    time.sleep(0.1)